# Build maps for covidsense data

We will build static, as well as time-series choropleths

In [ ]:
import os, sys, tqdm
import pprint
import datetime
from collections import Counter

import matplotlib.pyplot as plt
from matplotlib import cm, style

import seaborn as sns
import geopandas as gpd
import pandas as pd

import requests
import json

import folium
import branca
from folium.plugins import TimeSliderChoropleth

import numpy as np
from scipy import interpolate

from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

d2s = lambda x : datetime.datetime.fromtimestamp(x/1000.0).strftime('%B %d')

plt.rcParams.update({'font.size': 15})
style.use('seaborn-paper') #sets the size of the charts
style.use('ggplot')

# Some userful references
# Multiple layers: https://www.nagarajbhat.com/post/folium-visualization/
# Tooltip: https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJsonPopupAndTooltip.ipynb
# 

In [ ]:
if __name__ == '__main__':
    states_geojson_name = 'data/us-states.json'
    states_abbrv_name = 'data/us-state-abbrv.json'
    db_name = 'data/covidsense_db.json'
    
    # Load db and json data
    with open(db_name, 'rb') as f:
        db = json.load(f)
        
    # Load geojson data
    with open(states_geojson_name, 'rb') as f:
        states_geojson = json.load(f)
        
    # ... and abbreviations
    with open(states_abbrv_name, 'rb') as f:
        states_abbrv_dict = json.load(f)
        name2code = {state['name']:state['alpha-2'] for state in states_abbrv_dict}
        states_abbrv = pd.DataFrame.from_dict(states_abbrv_dict)
        
    code2name = {name2code[key]:key for key in name2code}
        
    states_gpd = gpd.GeoDataFrame.from_features(states_geojson, crs='EPSG:4326')
    
    # Merge geojson and abbreviation data
    states_gpd = states_gpd.merge(states_abbrv, how='left', left_on='name', right_on='name')
    

In [ ]:
    epoch1 = datetime.datetime(2020, 4, 1, 0, 0).timestamp()
    epoch2 = int(datetime.date.today().strftime('%s'))
    
    start_date = '2020-4-1'
    end_date = datetime.date.today().strftime('%Y-%m-%d')
    
    epochs = pd.date_range(start=start_date, end=end_date, freq='d').astype(int)//10**9
    epochs = np.array(epochs)

In [ ]:
    # Now build dataframe from db
    pd_dict = {'state': [],
               'QIDS': [],
               'CAMS-R': [],
               'country': [],
               'nresponses': [],
               'nparticipants': []}
    
    pd_db = pd.DataFrame(columns=['state', 'QIDS', 'CAMS-R', 'country', 'nresponses', 'nparticipants', 'Age',
                                  'Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism',
                                  'Imagination', 'Male', 'Female'])
    cnt = 0
    
    for key in tqdm.tqdm(db):
        if True:
            try:
                state = name2code[db[key]['state']]
            except KeyError:
                continue
                
            nresponses = len(db[key]['panels'])

            if len(db[key]['QIDS']) > 0:
                qids = list(db[key]['QIDS'].values())[0]
            else:
                qids = float('NaN')

            if 'CAMS-R' in db[key]:
                camsr = db[key]['CAMS-R']
            else:
                camsr = float('NaN')
                
            if 'IPIP' in db[key]:
                ipip = db[key]['IPIP']
                
            age = db[key]['age']
            
            male = 0
            female = 0
            if db[key]['gender'] == 'male':
                male = 1
            else:
                female = 1

            if 'state'  in db[key]:
                state = name2code[db[key]['state']]
            else:
                state = 'ID'
                
            # Now add to pandas dataframe
            row = [state, qids, camsr, 'US', nresponses, 1, age, ipip[0], 
                   ipip[1], ipip[2], ipip[3], ipip[4], male, female]
            pd_db.loc[cnt] = row
            cnt += 1
    

In [ ]:
    # Pandas allows you to do per column aggregation!
    d = {'QIDS': 'mean',
         'CAMS-R': 'mean',
         'nresponses': 'mean',
         'nparticipants': 'sum',
         'Extraversion': 'mean',
         'Agreeableness': 'mean',
         'Conscientiousness': 'mean',
         'Neuroticism': 'mean',
         'Imagination': 'mean',
         'Age': 'mean',
         'Male': 'sum',
         'Female': 'sum'}
    
    # We need to change dtype of some columns
    int_keys = ['nresponses', 'nparticipants', 'Extraversion', 'Agreeableness',
                'Conscientiousness', 'Neuroticism', 'Imagination', 'Age', 'Male', 'Female']
    for key in int_keys:
        pd_db[key] = pd_db[key].astype(float)
    
    db_state = pd_db.groupby('state').agg(d)
    db_state = states_gpd.merge(db_state, how='left', left_on='alpha-2', right_on='state')

In [ ]:
    # Now create a new colormap
    colormap = branca.colormap.LinearColormap(
        vmin=db_state['QIDS'].quantile(0.0), 
        vmax=db_state['QIDS'].quantile(1), 
        colors=['red','orange', 'lightblue', 'blue'],
        caption="State Level Mean QIDS",
    )

In [ ]:
    # Time to create a new static map
    m = folium.Map(location=[35.3, -97.6], zoom_start=4)

    tooltip = GeoJsonTooltip(
        fields=["name", "nparticipants", "nresponses", "QIDS", 'CAMS-R', 'Age', 'Male', 'Female',
                'Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Imagination'],
        aliases=["Name:", "Participants:", "Average responses:", "QIDS:", 'CAMS-R:', 'Age:', 'Male:', 'Female:',
                 'Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Imagination'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 1px solid black;
            border-radius: 2px;
            box-shadow: 3px;
        """,
        max_width=800,
    )


    g = folium.GeoJson(
        db_state,
        style_function=lambda x: {
            "fillColor": colormap(x["properties"]["QIDS"])
            if x["properties"]["QIDS"] is not None
            else "transparent",
            "color": "black",
            "fillOpacity": 0.4,
        },
        tooltip=tooltip
    ).add_to(m)

    colormap.add_to(m)
    
    # Save the result
    m.save('results/us_static_by_state.html')
    
    m

In [ ]:
    # Build a QIDS only dictionary to plot time series of choropleths
    #qids_dict = {epoch:[] for epoch in epochs}
    #qids_dict['state'] = []
    
    mood_dict = {'terrible': 0,
                 'poor': 1,
                 'fair': 2,
                 'good': 3,
                 'great': 4,
                 'excellent': 5}
    
    qids_dict = {'state': [], 'qids': [], 'mood': []}
    
    for key in db:
        if 'state' in db[key] and db[key]['country'] == 'US' and\
        len(db[key]['QIDS']) > 0 and len(db[key]['A']) > 0:
            # First, extract and sort
            if len(db[key]['QIDS']) > 0:
                timestamps, vals = db[key]['QIDS'].keys(), db[key]['QIDS'].values()

                timestamps = np.array(list(timestamps)).astype(int)//1000
                vals = np.array(list(vals))

                if timestamps.size > 1:
                    indices = np.argsort(timestamps)

                    timestamps = timestamps[indices]
                    vals = vals[indices]

                    # Do 1D interpolation
                    func = interpolate.interp1d(timestamps, vals, kind='nearest',
                                                fill_value='extrapolate')

                    vals_interp = func(epochs)
                else:
                    vals_interp = np.ones_like(epochs)*vals
            else:
                vals_interp = np.ones_like(epochs)*np.nan
                
            if len(db[key]['A']) > 0:
                A_list = db[key]['A']
                timestamps = np.array([A_item['time'] for A_item in A_list])//1000
                vals = np.array([mood_dict[A_item['mood']] for A_item in A_list])
                
                if timestamps.size > 1:
                    indices = np.argsort(timestamps)

                    timestamps = timestamps[indices]
                    vals = vals[indices]

                    # Do 1D interpolation
                    func = interpolate.interp1d(timestamps, vals, kind='nearest',
                                                fill_value='extrapolate')

                    vals_interp_mood = func(epochs)
                else:
                    vals_interp_mood = np.ones_like(epochs)*vals
                    
            else:
                vals_interp_mood = np.ones_like(epochs)*np.nan
                
            # Assign
            try:
                qids_dict['state'].append(name2code[db[key]['state']])
            
                #for idx in range(len(epochs)):
                #    qids_dict[epochs[idx]].append(vals_interp[idx])
                qids_dict['qids'].append(vals_interp)
                qids_dict['mood'].append(vals_interp_mood)
            except KeyError:
                pass
    
    # Group by state
    # https://stackoverflow.com/questions/16975318/pandas-aggregate-when-column-contains-numpy-arrays
    agg_dict = {"qids": lambda x: list(x.mean()),
                'mood': lambda x: list(x.mean())}
    
    qids_pd = pd.DataFrame.from_dict(qids_dict)
    
    qids_state = qids_pd.groupby(by='state').agg(agg_dict)
    qids_state['qids'] = np.array(qids_state['qids'])
    qids_state['mood'] = np.array(qids_state['mood'])

In [ ]:
    # Create a custom color map for mood
    colormap = branca.colormap.LinearColormap(
        vmin=0, 
        vmax=5, 
        colors=['red','orange', 'blue'],
        caption="State Level Mean mood (0: Terrible, 5: Excellent)",
    )
    
    # Time to create these style dictionaries
    qids_dict = qids_state.to_dict()['mood']
    styledata = dict()
    
    states_json = states_gpd.to_json()
    
    state2id = {states_geojson['features'][idx]['id']:str(idx) for idx in range(len(states_geojson['features']))}
    
    # Good thing is, we have a colormap from our previous static map
    for key, val in qids_dict.items():
        tmp = {str(epochs[idx]):{'color': colormap(val[idx]), 'opacity':0.5} for idx in range(len(epochs))}
        styledata[state2id[key]] = tmp
        

In [ ]:
    # Final stage, assemble into a map
    m = folium.Map(location=[35.3, -97.6], zoom_start=4)
    
    g = TimeSliderChoropleth(
        states_json,
        styledict=styledata,
    ).add_to(m)

    colormap.add_to(m)
    
    m.save('results/us-qids-slider.html')
    
    m

In [ ]:
    qdict = qids_state.to_dict()
    
    state = 'PA'
    
    fig, ax1 = plt.subplots(dpi=150)

    ax2 = ax1.twinx()
    ax1.plot(epochs, qdict['mood'][state], 'g-')
    ax2.plot(epochs, qdict['qids'][state], 'b-')

    ax1.set_xlabel('Day')
    ax1.set_ylabel('Mood', color='g')
    ax2.set_ylabel('QIDS', color='b')

    plt.xlabel('Day')
    #plt.xticks(epochs[::7], labels=[d2s(epoch) for epoch in epochs[::7]*1000], rotation='vertical')
    ax1.set_xticks(epochs[::7])
    ax1.set_xticklabels([d2s(epoch) for epoch in epochs[::7]*1000], rotation='vertical')
    
    plt.title(code2name[state])
    plt.savefig('results/%s_qids_mood.svg'%code2name[state])
    plt.show()